In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:],scene['v_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])     
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:],scene['v_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 4,
                 embedding_size = 16,
                 hidden_size = 256):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=16, hidden_size=256, output_size=4):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.linear1(x)
        embedded = F.relu(x)
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(iterator):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #output whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :]
        
        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden
#         print("DECODER INPUT", decoder_input.shape)
#         print("DECODER HIDDEN", decoder_hidden[0].shape)
        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :4], target[:, di, :4]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        if counter % (0.1*len(train_loader)) == 0:
            print("10% passed")
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=4)
decoder = DecoderRNN(output_size=4)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 2

for epoch in range(1, num_epoch + 1):
    train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :4]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 4)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
25085,"[582.8023681640625, 1377.97705078125, -0.60831...","[582.2620239257812, 1378.2078857421875, -1.192...","[581.90673828125, 1378.6151123046875, -1.43625...","[582.002685546875, 1378.82470703125, -1.643224...","[581.8129272460938, 1378.9188232421875, -1.756...","[581.79296875, 1379.5152587890625, -1.97931253...","[581.4295043945312, 1379.708984375, -2.0306544...","[581.1984252929688, 1380.2864990234375, -2.126...","[580.8214111328125, 1380.4268798828125, -2.252...","[580.6307373046875, 1380.657470703125, -2.3662...",...,"[576.9500122070312, 1384.57958984375, -3.43515...","[576.3668823242188, 1384.8238525390625, -3.634...","[576.0175170898438, 1384.9466552734375, -3.782...","[575.7005004882812, 1384.9329833984375, -3.892...","[575.4264526367188, 1385.1136474609375, -3.984...","[575.1434936523438, 1385.5145263671875, -4.057...","[574.73095703125, 1385.750732421875, -4.108998...","[574.172119140625, 1385.8963623046875, -4.1360...","[573.7119750976562, 1385.993896484375, -4.1729...","[573.506591796875, 1386.0185546875, -4.2187867..."
13042,"[2148.707275390625, 691.6826782226562, 9.70230...","[2148.820556640625, 691.8534545898438, 9.07496...","[2149.98291015625, 692.8428344726562, 8.763030...","[2151.001953125, 693.6316528320312, 8.82927322...","[2151.741455078125, 694.378173828125, 8.770567...","[2152.4736328125, 695.0081176757812, 8.7874393...","[2153.31591796875, 695.5529174804688, 8.810082...","[2153.92041015625, 696.2648315429688, 8.701579...","[2154.740478515625, 696.78515625, 8.7852258682...","[2155.612548828125, 697.6026611328125, 8.87560...",...,"[2166.78662109375, 706.970458984375, 8.8946027...","[2167.62060546875, 707.9295043945312, 8.931816...","[2168.930419921875, 708.9660034179688, 8.95079...","[2170.005615234375, 709.7876586914062, 8.93860...","[2170.4892578125, 710.3733520507812, 8.9125757...","[2171.1025390625, 711.19677734375, 8.885819435...","[2171.771484375, 712.0203857421875, 8.87280178...","[2172.52001953125, 712.3224487304688, 8.827715...","[2173.47314453125, 712.8978271484375, 8.729709...","[2174.33056640625, 714.2573852539062, 8.717721..."
27714,"[2040.4136962890625, 615.5549926757812, -8.781...","[2039.1495361328125, 614.6051635742188, -10.22...","[2037.7864990234375, 613.313720703125, -10.376...","[2036.8857421875, 612.3871459960938, -10.37162...","[2035.970947265625, 611.7313232421875, -10.385...","[2034.91357421875, 611.0835571289062, -10.3007...","[2033.7021484375, 610.2581787109375, -10.27315...","[2032.6793212890625, 609.2830200195312, -10.38...","[2031.3829345703125, 608.32177734375, -10.3734...","[2030.016845703125, 607.2151489257812, -10.323...",...,"[2018.7720947265625, 598.3433227539062, -10.25...","[2017.95068359375, 597.712158203125, -10.22193...","[2017.5118408203125, 596.5943603515625, -10.15...","[2017.047607421875, 595.2156982421875, -10.030...","[2015.9969482421875, 594.5222778320312, -9.885...","[2014.506103515625, 593.8883666992188, -9.7789...","[2013.4036865234375, 592.95947265625, -9.77730...","[2012.296630859375, 591.9972534179688, -9.8239...","[2011.031005859375, 590.9373779296875, -9.7607...","[2009.919189453125, 590.0933227539062, -9.5560..."
14255,"[1975.16845703125, 662.1294555664062, -7.27635...","[1974.3153076171875, 661.5044555664062, -7.649...","[1973.5020751953125, 660.7239379882812, -7.817...","[1972.6944580078125, 659.9080810546875, -7.691...","[1971.9244384765625, 659.2512817382812, -7.813...","[1971.167724609375, 658.689697265625, -7.82785...","[1970.2457275390625, 658.2103271484375, -7.697...","[1969.5816650390625, 657.62548828125, -7.65787...","[1969.12744140625, 656.8045043945312, -7.55374...","[1968.254150390625, 656.2335815429688, -7.4962...",...,"[1959.8194580078125, 648.1607055664062, -7.576...","[1958.7734375, 647.6698608398438, -7.626999378...","[1957.986572265625, 647.2095336914062, -7.6091...","[1957.2930908203125, 646.7346801757812, -7.582...","[1956.553955078125, 645.6915893554688, -7.5828...","[195

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)[[0, 1]]

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
25085,582.802368,1377.977051,582.262024,1378.207886,581.906738,1378.615112,582.002686,1378.824707,581.812927,1378.918823,...,575.143494,1385.514526,574.730957,1385.750732,574.172119,1385.896362,573.711975,1385.993896,573.506592,1386.018555
13042,2148.707275,691.682678,2148.820557,691.853455,2149.982910,692.842834,2151.001953,693.631653,2151.741455,694.378174,...,2171.102539,711.196777,2171.771484,712.020386,2172.520020,712.322449,2173.473145,712.897827,2174.330566,714.257385
27714,2040.413696,615.554993,2039.149536,614.605164,2037.786499,613.313721,2036.885742,612.387146,2035.970947,611.731323,...,2014.506104,593.888367,2013.403687,592.959473,2012.296631,591.997253,2011.031006,590.937378,2009.919189,590.093323
14255,1975.168457,662.129456,1974.315308,661.504456,1973.502075,660.723938,1972.694458,659.908081,1971.924438,659.251282,...,1955.777832,644.889465,1955.306274,644.433594,1954.863892,643.795166,1953.935547,643.060364,1953.103027,642.543823
17910,2168.818359,711.693665,2168.986084,712.456116,2169.379883,712.923584,2169.861572,712.900940,2170.479004,713.583008,...,2180.536377,723.350464,2180.811768,723.630066,2181.042969,723.863281,2181.218018,724.027222,2181.467285,724.251099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35151,416.518585,1125.160400,416.279449,1125.753784,416.004089,1125.966553,416.206604,1126.390625,416.142181,1126.700439,...,414.268127,1138.100220,414.070892,1138.463379,413.902344,1138.946289,413.704010,1139.247681,413.867126,1139.815918
34017,583.500732,1451.452271,582.966003,1450.280884,583.705566,1449.174683,583.353577,1448.609863,583.466248,1447.671997,...,582.875732,1427.321777,582.925415,1426.276367,582.814880,1425.261963,582.725159,1424.506592,582.779785,1423.757202
44,421.015411,1016.848816,420.837646,1017.751221,420.742157,1018.980591,420.543671,1019.701965,420.598541,1020.394470,...,419.343872,1040.859497,419.700867,1042.053833,419.621521,1043.397217,419.650391,1045.082153,419.678894,1046.508057


In [12]:
df2.to_csv("outputs2epb4_v_all_decoderhidden256.csv", index=True)